# Prepare the data 

In [1]:
import sys,os
import numpy as np
import random
import pandas as pd

In [2]:
file = 'D:\Evans microbial community\data_for_pipeline\metadata_full_MMPRNT_G5.LC_LUX_trunc_rar_016017_v2.txt'
#label = 'D:\Evans microbial community\data_for_pipeline\ELSA_module_016017_LUX_OTU_sum_MMPRNT_G5_LC_LUX_016017.txt'
label = 'D:\Evans microbial community\data_for_pipeline\Rarefied_diversity_MMPRNT_G5_LC_LUX_016017.txt'
site = 'LUX'
site_not_used = 'LC'
cv_num = 10
test_size = 0.1 ### what the proportion of your data you want to hold out as test set, 
                ### which will never be seen when you train the model
feature_2_onehotencoding = ['FertStatus','thermal_two_year','thermal_2019','thermal_2018']
ML_method = 'RandomForestRegressor'
short_name = 'Multioutput_regression_richness_randomized'
save_model = '%s_%s.sav'%(short_name,ML_method)
save_parameter = '%s_%s_parameter.txt'%(short_name,ML_method)
save_performance = '%s_%s_performance.txt'%(short_name,ML_method)
save_imp = '%s_%s_feature_importance.txt'%(short_name,ML_method)
randomized = 'True' # or 'False'. Note that if randomized = 'True', don't forget to change the short_name, in case your previous results would be overwrote.

In [3]:
df = pd.read_csv(file, sep='\t', index_col = 0, header = 0)
if 'ELSA' in label:
    df_lable = pd.read_csv(label, sep='\t', index_col = 42, header = 0)
if 'Rarefied' in label:
    df_lable = pd.read_csv(label, sep='\t', index_col = 0, header = 0)
ML_matrix = pd.concat([df_lable,df], axis=1, sort = False)

In [4]:
df_target_site = ML_matrix.loc[ML_matrix['siteID']==site]
df_target_site = df_target_site.drop(["collectionDate","siteID",\
                                             "UTM_Lat_Cord","UTM_Lon_Cord"],axis=1)

df_other_site = ML_matrix.loc[ML_matrix['siteID']==site_not_used]
df_other_site = df_other_site.drop(["collectionDate","siteID",\
                                             "UTM_Lat_Cord","UTM_Lon_Cord"],axis=1)

df_target_site["thermal_two_year"].isna().sum()

22

# Split the data to traning and testing

In [5]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df_target_site, \
                                       test_size=test_size, random_state=42)
X_train = train_set.drop(df_lable.columns, axis=1) 
X_test = test_set.drop(df_lable.columns, axis=1)
X_on_test_site = df_other_site.drop(df_lable.columns, axis=1)

y_train = train_set[df_lable.columns]
y_test = test_set[df_lable.columns]
y_on_test_site = df_other_site[df_lable.columns]

# OneHotEncoding, handle with NaN data (keep them as NaN)


#### Fit on training

In [6]:
from sklearn.preprocessing import OneHotEncoder
def OneHotEncoder_fit_transform(df,feature_2_onehotencoding):
    new_columns = []
    Onehot = {}
    col_2_1hot = df.loc[:,feature_2_onehotencoding]
    for col in feature_2_onehotencoding:
        Onehot[col] = OneHotEncoder(sparse=False, handle_unknown='ignore')
        c_1hot_use = pd.DataFrame(col_2_1hot.loc[col_2_1hot.loc[:,col].notna(),col])
        c_1hot_na = pd.DataFrame(col_2_1hot.loc[col_2_1hot.loc[:,col].isna(),col])
        if c_1hot_na.shape[0] == 0:
            c_1hot_encoded = pd.DataFrame(Onehot[col].fit_transform(c_1hot_use))
            c_1hot_encoded.columns = [col + '_' + '%s'%sub for sub in Onehot[col].categories_[0]]
            c_1hot_encoded.index = col_2_1hot.index
            for columns in c_1hot_encoded.columns:
                new_columns.append(columns)
        if c_1hot_na.shape[0] != 0:
            c_1hot_encoded = pd.DataFrame(Onehot[col].fit_transform(c_1hot_use))
            c_1hot_encoded.columns = [col + '_' + '%s'%sub for sub in Onehot[col].categories_[0]]
            c_1hot_encoded.index = c_1hot_use.index
            for columns in c_1hot_encoded.columns:
                c_1hot_na[columns] = np.nan
                new_columns.append(columns)
            c_1hot_na = c_1hot_na.drop(col,axis=1)
            c_1hot_encoded = pd.concat([c_1hot_encoded,c_1hot_na],axis=0)
        df = pd.concat([df,c_1hot_encoded],axis=1)
    return(df,Onehot,new_columns)

#### transform on test

In [7]:
def OneHotEncoder_transform(df,feature_2_onehotencoding,Onehot):
    col_2_1hot = df.loc[:,feature_2_onehotencoding]
    for col in feature_2_onehotencoding:
        c_1hot_use = pd.DataFrame(col_2_1hot.loc[col_2_1hot.loc[:,col].notna(),col])
        c_1hot_na = pd.DataFrame(col_2_1hot.loc[col_2_1hot.loc[:,col].isna(),col])
        if c_1hot_na.shape[0] == 0:
            c_1hot_encoded = pd.DataFrame(Onehot[col].transform(c_1hot_use))
            c_1hot_encoded.columns = [col + '_' + '%s'%sub for sub in Onehot[col].categories_[0]]
            c_1hot_encoded.index = col_2_1hot.index
            df = pd.concat([df,c_1hot_encoded],axis=1)
        if c_1hot_na.shape[0] != 0 and c_1hot_use.shape[0] != 0:
            c_1hot_encoded = pd.DataFrame(Onehot[col].transform(c_1hot_use))
            c_1hot_encoded.columns = [col + '_' + '%s'%sub for sub in Onehot[col].categories_[0]]
            c_1hot_encoded.index = c_1hot_use.index
            for columns in c_1hot_encoded.columns:
                c_1hot_na[columns] = np.nan
            c_1hot_na = c_1hot_na.drop(col,axis=1)
            c_1hot_encoded = pd.concat([c_1hot_encoded,c_1hot_na],axis=0)
            df = pd.concat([df,c_1hot_encoded],axis=1)
        if c_1hot_use.shape[0] == 0:
            columns = [col + '_' + '%s'%sub for sub in Onehot[col].categories_[0]]
            for column in columns:
                c_1hot_na[column] = np.nan
            c_1hot_na = c_1hot_na.drop(col,axis=1)  
            df = pd.concat([df,c_1hot_na],axis=1)
    return(df)    

In [8]:
X_train, Onehot, new_columns = OneHotEncoder_fit_transform(X_train,feature_2_onehotencoding)
X_test = OneHotEncoder_transform(X_test,feature_2_onehotencoding,Onehot)
X_on_test_site = OneHotEncoder_transform(X_on_test_site,feature_2_onehotencoding,Onehot)

D:\softinstall\Anaconda3\envs\tf2\lib\site-packages\ipykernel_launcher.py:25: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

D:\softinstall\Anaconda3\envs\tf2\lib\site-packages\ipykernel_launcher.py:25: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

D:\softinstall\Anaconda3\envs\tf2\lib\site-packages\ipykernel_launcher.py:25: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 

### drop the original columns

In [9]:
X_train = X_train.drop(feature_2_onehotencoding,axis=1)
X_test = X_test.drop(feature_2_onehotencoding,axis=1)
X_on_test_site = X_on_test_site.drop(feature_2_onehotencoding,axis=1)

# Impute missing data using KNN, five Ks

### 1. drop features with >50% missing data

In [10]:
Miss_count = X_train.count(0)
Col_to_drop = Miss_count[Miss_count <= 0.5*X_train.shape[0]].index.tolist()
Col_to_drop
X_train.drop(Col_to_drop,axis=1,inplace=True)
X_test.drop(Col_to_drop,axis=1,inplace=True)
X_on_test_site.drop(Col_to_drop,axis=1,inplace=True)

### 2. impute

In [11]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import KNNImputer
class KNNImputer_Ks(BaseEstimator, TransformerMixin):
    def __init__(self, *Ks):
        self.Ks = Ks
    def fit(self, X,Ks):
        D_imputer = {}        
        for k in [3,4,5,6,7]:
            imputer = KNNImputer(n_neighbors=k)
            D_imputer[k] = imputer.fit(X)              
        return D_imputer
    def transform(self, X):
        Impute_train = {}
        for k in [3,4,5,6,7]:
            Impute_train[k] = pd.DataFrame(D_imputer[k].transform(X))
            Impute_train[k].index = X.index
            Impute_train[k].columns = X.columns 
            if k == 3:
                Imputed = Impute_train[k].copy(deep=True)
                Imputed.loc[:,:] = 0
            Imputed = Imputed.add(Impute_train[k],fill_value=0)
        return Imputed/5

In [12]:
imputer_knn = KNNImputer_Ks()
D_imputer = imputer_knn.fit(X_train, Ks="3,4,5,6,7")
X_train_KNN = imputer_knn.transform(X_train)
X_test_KNN = imputer_knn.transform(X_test)
X_on_test_site_KNN =  imputer_knn.transform(X_on_test_site)

### round the imputed values for binary features

In [13]:
for col in new_columns:
    X_train_KNN[col] = round(X_train_KNN[col],0)
    X_test_KNN[col] = round(X_test_KNN[col],0)
    X_on_test_site_KNN[col] = round(X_on_test_site_KNN[col],0)

# Shuffle the y_train, y_test to get the backgound, or performance of random guessing


In [ ]:
from numpy import random
from numpy.random import permutation
if randomized == 'True':
    for i in range(0,y_train.shape[1]):
        y_train.iloc[:,i] = np.random.permutation(y_train.iloc[:,i])
        y_test.iloc[:,i] = np.random.permutation(y_test.iloc[:,i])
        y_on_test_site.iloc[:,i] = np.random.permutation(y_on_test_site.iloc[:,i])

# Machine learning

## RandomForestRegressor

### 1. GridSearch

In [24]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score

In [25]:
if y_train.shape[1] == 1:
    param_grid = {'max_depth':[3, 5, 10], \
              'max_features': [0.1, 0.5, 'sqrt', 'log2', None], \
              'n_estimators': [10, 100,500,1000]}
    grid_search = GridSearchCV(RandomForestRegressor(random_state=42), param_grid, cv=cv_num, \
                           scoring='neg_mean_squared_error', verbose=2,n_jobs=5)
    grid_search.fit(X_train_KNN, y_train)

In [ ]:
if y_train.shape[1] > 1:    
    gsc = GridSearchCV(
                estimator=RandomForestRegressor(random_state=42),
                param_grid={'max_depth':[3, 5, 10], \
                  'max_features': [0.1, 0.5, 'sqrt', 'log2', None], \
                  'n_estimators': [10, 100,500,1000]},    
                cv=cv_num, scoring='neg_mean_squared_error', verbose=2, n_jobs=5)
    grid_search = MultiOutputRegressor(gsc).fit(X_train_KNN, y_train)

Fitting 10 folds for each of 60 candidates, totalling 600 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  31 tasks      | elapsed:   22.7s
[Parallel(n_jobs=5)]: Done 152 tasks      | elapsed:   51.6s
[Parallel(n_jobs=5)]: Done 355 tasks      | elapsed:  2.0min
[Parallel(n_jobs=5)]: Done 600 out of 600 | elapsed:  4.5min finished


Fitting 10 folds for each of 60 candidates, totalling 600 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  31 tasks      | elapsed:    7.0s
[Parallel(n_jobs=5)]: Done 152 tasks      | elapsed:   37.8s
[Parallel(n_jobs=5)]: Done 355 tasks      | elapsed:  1.8min


### 2. Predict

In [ ]:
def PCC_multioutput(y,pred):
    R = []
    for i in range(0,y.shape[1]):
        R.append(np.corrcoef(np.array(y.iloc[:,i]), np.array(pred.iloc[:,i]))[0,1])
                 
    return (np.array(R))

    # mse: Mean squared error regression loss
    # evs: Explained variance regression score
    # r2: (coefficient of determination) regression score. Best possible score is 1.0 and it can 
    be negative (because the model can be arbitrarily worse). A constant model that always 
    predicts the expected value of y, 
    # disregarding the input features, would get a R^2 score of 0.0.
    # cor: Pearson Correlation Coefficient between true y and predicted y

In [ ]:
if y_train.shape[1] == 1:
    cv_pred = cross_val_predict(estimator=grid_search.best_estimator_, X=X_train_KNN, \
                                y=y_train, cv=cv_num)
    cv_mse = mean_squared_error(y_train, cv_pred)
    cv_evs = explained_variance_score(y_train, cv_pred)
    cv_r2 = r2_score(y_train, cv_pred)
    cv_cor = np.corrcoef(np.array(y_train), cv_pred)[0,1]
    
    pred_train = grid_search.best_estimator_.predict(X_train_KNN)
    train_mse = mean_squared_error(y_train, pred_train)
    train_evs = explained_variance_score(y_train, pred_train)
    train_r2 = r2_score(y_train, pred_train)
    train_cor = np.corrcoef(np.array(y_train), pred_train)[0,1]
    
    pred_test = grid_search.best_estimator_.predict(X_test_KNN)
    test_mse = mean_squared_error(y_test, pred_test)
    test_evs = explained_variance_score(y_test, pred_test)
    test_r2 = r2_score(y_test, pred_test)
    test_cor = np.corrcoef(np.array(y_test), pred_test)  
 
    pred_test = grid_search.best_estimator_.predict(X_on_test_site_KNN)
    test_site_mse = mean_squared_error(y_on_test_site, pred_on_test_site)
    test_site_evs = explained_variance_score(y_on_test_site, pred_on_test_site)
    test_site_r2 = r2_score(y_on_test_site, pred_on_test_site)
    test_site_cor = np.corrcoef(np.array(y_on_test_site), pred_on_test_site)[0,1]    

In [ ]:
def Mul_reg_cv_predict(Mul_estimators,X,y):
    pred = pd.DataFrame()
    for i in range(0,y.shape[1]):
        prediction = pd.DataFrame(cross_val_predict(estimator=Mul_estimators.estimators_[i].best_estimator_, \
                                              X=X, y=y.iloc[:,i], cv=cv_num))
        pred = pd.concat([pred,prediction],axis=1)
    pred.columns = y.columns
    pred.index = y.index
    return (pred)

In [ ]:
def Mul_reg_predict(Mul_estimators,X,y):
    pred = pd.DataFrame()
    for i in range(0,y.shape[1]):
        prediction = pd.DataFrame(Mul_estimators.estimators_[i].best_estimator_.predict(X))
        pred = pd.concat([pred,prediction],axis=1)
    pred.columns = y.columns
    pred.index = y.index
    return (pred)

In [70]:
if y_train.shape[1] > 1:
    cv_pred = Mul_reg_cv_predict(grid_search, X_train_KNN, y_train)
    cv_mse = mean_squared_error(y_train, cv_pred,multioutput='raw_values')
    cv_evs = explained_variance_score(y_train, cv_pred,multioutput='raw_values')
    cv_r2 = r2_score(y_train, cv_pred,multioutput='raw_values')
    cv_cor = PCC_multioutput(y_train, cv_pred)
    
    pred_train = Mul_reg_predict(grid_search, X_train_KNN, y_train)
    train_mse = mean_squared_error(y_train, pred_train,multioutput='raw_values')
    train_evs = explained_variance_score(y_train, pred_train,multioutput='raw_values')
    train_r2 = r2_score(y_train, pred_train,multioutput='raw_values')
    train_cor = PCC_multioutput(y_train, pred_train)    
    
     
    pred_test = Mul_reg_predict(grid_search, X_test_KNN, y_test)
    test_mse = mean_squared_error(y_test, pred_test,multioutput='raw_values')
    test_evs = explained_variance_score(y_test, pred_test,multioutput='raw_values')
    test_r2 = r2_score(y_test, pred_test,multioutput='raw_values')
    test_cor = PCC_multioutput(y_test, pred_test) 
    
    pred_on_test_site = Mul_reg_predict(grid_search, X_on_test_site_KNN, y_on_test_site)
    test_site_mse = mean_squared_error(y_on_test_site, pred_on_test_site,multioutput='raw_values')
    test_site_evs = explained_variance_score(y_on_test_site, pred_on_test_site,multioutput='raw_values')
    test_site_r2 = r2_score(y_on_test_site, pred_on_test_site,multioutput='raw_values')
    test_site_cor = PCC_multioutput(y_on_test_site, pred_on_test_site)    
    

# Parse the results

### Save the fitted model

In [71]:
import pickle
if y_train.shape[1] == 1:
    pickle.dump(grid_search.best_estimator_, open(save_model, 'wb'))
else:
    pickle.dump(grid_search, open(save_model, 'wb'))

### write the hyperparameters

In [74]:
out = open(save_parameter,'w')
out.write('The model is built using data from %s, and applied to %s and %s.\n\n'%(site,site,site_not_used))
out.write('There are %s training instances.\n'%X_train_KNN.shape[0])
out.write('There are %s test instances.\n'%X_test_KNN.shape[0])
out.write('There are %s instances in the other site.\n'%X_on_test_site_KNN.shape[0])
out.write('There are %s feature used\n\n'%X_train_KNN.shape[1])
if y_train.shape[1] == 1:
    out.write('The model is built using %s, with:\n'%ML_method)
    for key in grid_search.best_params_:
        out.write('\t%s: %s\n'%(key,grid_search.best_params_[key]))

if y_train.shape[1] > 1:
    out.write('The model is built using %s, multioutput, with:\n'%ML_method)
    for i in range(0, y_train.shape[1]):
        g = grid_search.estimators_[i].best_params_
        out.write('\t\t%s:\n'%y_train.columns[i])
        for key in g:
            out.write('\t\t\t%s: %s\n'%(key,g[key]))
            
out.close()

### write performance of models

In [80]:
if y_train.shape[1] == 1:
    out = open(save_performance,'w')
    out.write('Prediction\tmse\tevs\tr2\tPCC\n')
    out.write('CV\t%s\t%s\t%s\t%s\n'%(cv_mse,cv_evs,cv_r2,cv_cor))
    out.write('Train\t%s\t%s\t%s\t%s\n'%(train_mse,train_evs,train_r2,train_cor))
    out.write('Test\t%s\t%s\t%s\t%s\n'%(test_mse,test_evs,test_r2,test_cor))
    out.write('Other_site\t%s\t%s\t%s\t%s\n\n'%(test_site_mse,test_site_evs,test_site_r2,test_site_cor))
    out.close()
    
if y_train.shape[1] > 1:
    perf = pd.DataFrame([cv_mse,cv_evs,cv_r2,cv_cor,train_mse,train_evs,train_r2,train_cor,\
                         test_mse,test_evs,test_r2,test_cor,test_site_mse,test_site_evs,\
                         test_site_r2,test_site_cor])
    perf.columns = y_train.columns
    perf.index = ['cv_mse','cv_evs','cv_r2','cv_cor','train_mse','train_evs','train_r2','train_cor',\
                         'test_mse','test_evs','test_r2','test_cor','test_site_mse','test_site_evs',\
                         'test_site_r2','test_site_cor']
    perf.to_csv(save_performance,index=True, header=True,sep="\t")

### write important features

In [84]:
def Imp_dir(X, y, imp):
    for i in range(0,imp.shape[0]):
        feature = imp.iloc[i,0]
        pcc = np.corrcoef(np.array(X[feature]), np.array(y))[0,1]
        if pcc < 0:
            imp.iloc[i,1] = -1 * imp.iloc[i,1]
    return (imp)

In [93]:
if y_train.shape[1] == 1:
    imp = pd.DataFrame({'Feature':X_train_KNN.columns, 'Importance':\
                        grid_search.best_estimator_.feature_importances_})
    imp_sorted = imp.sort_values(by='Importance', ascending=False)
    imp_sorted_dir = Imp_dir(X_train_KNN, y_train, imp_sorted)
    imp_sorted_dir.to_csv(save_imp, index=False, header=True,sep="\t")
    
if y_train.shape[1] > 1:
    for i in range(0,y_train.shape[1]):
        imp = pd.DataFrame({'Feature':X_train_KNN.columns, 'Importance_%s'%y_train.columns[i]:\
                        grid_search.estimators_[i].best_estimator_.feature_importances_}) 
        imp_dir = Imp_dir(X_train_KNN, y_train.iloc[:,i], imp)
        if i == 0:
            Imp = imp_dir
        else:
            Imp = pd.concat([Imp,imp_dir.iloc[:,1]],axis=1)    
    Imp.to_csv(save_imp,index=False, header=True,sep="\t")